In [1]:
import pandas as pd
import gzip
import os
import numpy as np

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_sellout = "datasets/tb_sellout_02.txt.gz"
arch_maestro_prod = "datasets/maestro_productos_depurado.csv"
arch_exogenas = "datasets/emp3_exogenas.csv"
arch_salida_completo = "datasets/emp3_sellout_base.csv"
arch_salida_producto = "datasets/emp3_sellout_producto_base.csv"
arch_salida_cliente = "datasets/emp3_sellout_cliente_base.csv"
arch_salida_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_salida_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_salida_cat3 = "datasets/emp3_sellout_cat3_base.csv"
arch_salida_estrellas = "datasets/emp3_sellout_base_estrellas.csv"

In [4]:
def diferencia_meses(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [5]:
clientes_estrella = [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013]

## Sellout

In [6]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [7]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [8]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

2945818
2945818


**--> sin duplicados**

In [9]:
df_sellout.isna().sum()

periodo                  0
customer_id              0
product_id               0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
dtype: int64

**--> sin nulos**

In [10]:
df_sellout.periodo.unique()                

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [11]:
len(df_sellout.product_id.unique())          

1233

--> algunos productos no van a tener descripción

In [12]:
len(df_sellout.customer_id.unique())            

597

In [13]:
df_sellout.plan_precios_cuidados.unique()          

array([0, 1])

In [14]:
df_sellout[df_sellout.tn==0]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


--> si no hay ventas, no hay registro en 0 (directamente no hay registro)

In [15]:
df_sellout.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [16]:
# Como control, sumo tns
tn_suma_original = round(sum(df_sellout.tn))
print("Toneladas Total Control:", round(sum(df_sellout.tn),0))

Toneladas Total Control: 1324989.0


## Completado de Períodos sin Ventas

* Los pares <cliente,producto> que no aparecen un mes van a ser completados con un registro en 0
* Sin embargo, se completará a partir de su primer mes (y no para atrás)
* Esto aplicará para el primer mes del cliente y del producto

In [17]:
periodos = df_sellout.periodo.unique()
cant_periodos = len(periodos)
periodos, cant_periodos

(array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
        201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
        201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
        201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
        201909, 201910, 201911, 201912]),
 36)

In [18]:
productos = df_sellout.product_id.unique()
cant_productos = len(productos)
cant_productos

1233

In [19]:
clientes = df_sellout.customer_id.unique()
cant_clientes = len(clientes)
cant_clientes

597

In [20]:
len(df_sellout),cant_productos*cant_periodos*cant_clientes

(2945818, 26499636)

**--> no todos los productos y clientes están en todos los períodos (2.95M vs 26.5M)**

In [21]:
# Obtengo el primer mes de cada producto
df_primer_mes_prod = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_prod = df_primer_mes_prod.rename(columns={"periodo":"primer_periodo_prod"})
df_primer_mes_prod.tail()

,product_id,primer_periodo_prod
1228,21295,201701
1229,21296,201708
1230,21297,201701
1231,21298,201708
1232,21299,201708


In [22]:
# Obtengo el primer mes de cada cliente
df_primer_mes_cliente = df_sellout.groupby("customer_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_cliente = df_primer_mes_cliente.rename(columns={"periodo":"primer_periodo_cliente"})
df_primer_mes_cliente.tail()

,customer_id,primer_periodo_cliente
592,10633,201701
593,10634,201702
594,10635,201701
595,10636,201702
596,10637,201709


In [23]:
# Se va a poner 0 a todos los periodos donde el producto no se vendio
df_cartesiano = pd.DataFrame(data={"product_id":productos}).merge(pd.DataFrame(data={"periodo":periodos}), how='cross')
df_cartesiano = df_cartesiano.merge(pd.DataFrame(data={"customer_id":clientes}), how='cross')

df_cartesiano["cero_ventas"] = 0
len(df_cartesiano)

26499636

In [24]:
df_cartesiano.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26499636 entries, 0 to 26499635
Data columns (total 4 columns):
 #   Column       Dtype
---  ------       -----
 0   product_id   int64
 1   periodo      int64
 2   customer_id  int64
 3   cero_ventas  int64
dtypes: int64(4)
memory usage: 1010.9 MB


In [25]:
df_cartesiano.head()

,product_id,periodo,customer_id,cero_ventas
0,20524,201701,10234,0
1,20524,201701,10032,0
2,20524,201701,10217,0
3,20524,201701,10125,0
4,20524,201701,10012,0


In [26]:
df_sellout_complet = df_cartesiano.merge(df_sellout, how="left",on=["product_id","customer_id","periodo"])
len(df_sellout_complet)

26499636

In [27]:
df_sellout_complet.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,20524,201701,10234,0,0.0,2.0,0.05300,0.05300
1,20524,201701,10032,0,0.0,1.0,0.13628,0.13628
2,20524,201701,10217,0,0.0,1.0,0.03028,0.03028
3,20524,201701,10125,0,0.0,1.0,0.02271,0.02271
4,20524,201701,10012,0,0.0,11.0,1.54452,1.54452


In [28]:
df_sellout_complet.isna().sum()

product_id                      0
periodo                         0
customer_id                     0
cero_ventas                     0
plan_precios_cuidados    23553818
cust_request_qty         23553818
cust_request_tn          23553818
tn                       23553818
dtype: int64

In [29]:
# Imputo
df_sellout_complet.cero_ventas = np.where(df_sellout_complet.tn.isna(),1,0)
df_sellout_complet.tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.tn)
df_sellout_complet.cust_request_tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_tn)
df_sellout_complet.cust_request_qty = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_qty)
df_sellout_complet.plan_precios_cuidados = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.plan_precios_cuidados)

In [30]:
df_sellout_complet.isna().sum()

product_id               0
periodo                  0
customer_id              0
cero_ventas              0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
dtype: int64

In [31]:
df_sellout_complet.cero_ventas.sum(),len(df_cartesiano)-len(df_sellout)

(23553818, 23553818)

In [32]:
# Ahora, cruzo con el primer mes (para luego borrar aquellos registros previos al primer mes de datos)
print(len(df_sellout_complet))
df_sellout_complet_desde_1er_mes = df_sellout_complet.merge(df_primer_mes_prod,on="product_id",how="inner")
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.merge(df_primer_mes_cliente,on="customer_id",how="inner")
print(len(df_sellout_complet_desde_1er_mes))

26499636
26499636


In [33]:
# Borro los previos al primer mes
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_prod) & (df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_cliente)]

print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.tail(10)

19639107


,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,primer_periodo_prod,primer_periodo_cliente
26499311,20992,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499347,21018,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499383,20993,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499419,20764,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499455,21026,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499491,21054,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499527,20728,201912,10572,1,0.0,0.0,0.0,0.0,201911,201912
26499563,20792,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912
26499599,20854,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912
26499635,20770,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912


--> **Quedan 19.6M de registros**

### --> La que sigue es la parte que más demora...

In [34]:
df_sellout_complet_desde_1er_mes['periodo_fecha'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['periodo'], format='%Y%m')

# Se agrega una variable que tenga el mes y tambien la cantidad de meses de historia del producto
df_sellout_complet_desde_1er_mes["mes"] = pd.DatetimeIndex(df_sellout_complet_desde_1er_mes.periodo_fecha).month
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_prod'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_prod'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_prod"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_prod"]),axis=1)
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_cliente'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_cliente'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_cliente"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_cliente"]),axis=1)

#Se agrega columna de "cliente_estrella" y par producto-cliente
df_sellout_complet_desde_1er_mes["cliente_estrella"]=np.where(df_sellout_complet_desde_1er_mes.customer_id.isin(clientes_estrella),1,0)
df_sellout_complet_desde_1er_mes["prod_cust"] = df_sellout_complet_desde_1er_mes.product_id.astype(str) + "-" + df_sellout_complet_desde_1er_mes.customer_id.astype(str)
# Dejo unicamente las columnas de meses_historia
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.drop(columns=["primer_periodo_prod","primer_periodo_cliente","primer_periodo_fecha_prod","primer_periodo_fecha_cliente"])

In [35]:
df_sellout_complet_desde_1er_mes.head(10)

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,meses_historia_prod,meses_historia_cliente,cliente_estrella,prod_cust
0,20524,201701,10234,0,0.0,2.0,0.05300,0.05300,2017-01-01,1,0,0,0,20524-10234
1,20524,201702,10234,1,0.0,0.0,0.00000,0.00000,2017-02-01,2,1,1,0,20524-10234
2,20524,201703,10234,0,0.0,1.0,0.01514,0.01514,2017-03-01,3,2,2,0,20524-10234
3,20524,201704,10234,1,0.0,0.0,0.00000,0.00000,2017-04-01,4,3,3,0,20524-10234
4,20524,201705,10234,1,0.0,0.0,0.00000,0.00000,2017-05-01,5,4,4,0,20524-10234
5,20524,201706,10234,0,0.0,1.0,0.03786,0.03786,2017-06-01,6,5,5,0,20524-10234
6,20524,201707,10234,1,0.0,0.0,0.00000,0.00000,2017-07-01,7,6,6,0,20524-10234
7,20524,201708,10234,1,0.0,0.0,0.00000,0.00000,2017-08-01,8,7,7,0,20524-10234
8,20524,201709,10234,1,0.0,0.0,0.00000,0.00000,2017-09-01,9,8,8,0,20524-10234
9,20524,201710,10234,1,0.0,0.0,0.00000,0.00000,2017-10-01,10,9,9,0,20524-10234


### Comprobación de casos puntuales

In [36]:
df_sellout[(df_sellout.customer_id==10637)&(df_sellout.product_id==20741)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
776913,201709,10637,20741,0,1,0.00144,0.00144


In [37]:
df_sellout[(df_sellout.customer_id==10637)].periodo.min()

201709

In [38]:
df_sellout[(df_sellout.product_id==20741)].periodo.min()

201708

In [39]:
df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.customer_id==10637)&(df_sellout_complet_desde_1er_mes.product_id==20741)]

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,meses_historia_prod,meses_historia_cliente,cliente_estrella,prod_cust
23290748,20741,201709,10637,0,0.0,1.0,0.00144,0.00144,2017-09-01,9,1,0,0,20741-10637
23290749,20741,201710,10637,1,0.0,0.0,0.00000,0.00000,2017-10-01,10,2,1,0,20741-10637
23290750,20741,201711,10637,1,0.0,0.0,0.00000,0.00000,2017-11-01,11,3,2,0,20741-10637
23290751,20741,201712,10637,1,0.0,0.0,0.00000,0.00000,2017-12-01,12,4,3,0,20741-10637
23290752,20741,201801,10637,1,0.0,0.0,0.00000,0.00000,2018-01-01,1,5,4,0,20741-10637
23290753,20741,201802,10637,1,0.0,0.0,0.00000,0.00000,2018-02-01,2,6,5,0,20741-10637
23290754,20741,201803,10637,1,0.0,0.0,0.00000,0.00000,2018-03-01,3,7,6,0,20741-10637
23290755,20741,201804,10637,1,0.0,0.0,0.00000,0.00000,2018-04-01,4,8,7,0,20741-10637
23290756,20741,201805,10637,1,0.0,0.0,0.00000,0.00000,2018-05-01,5,9,8,0,20741-10637
23290757,20741,201806,10637,1,0.0,0.0,0.00000,0.00000,2018-06-01,6,10,9,0,20741-10637


**--> se generó OK, desde 201709 en adelante únicamente**

## Incorporo Maestro y Exógenas

In [40]:
df_maestro_prod = pd.read_csv(arch_maestro_prod)

In [41]:
df_maestro_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cat1               1251 non-null   object
 1   cat2               1251 non-null   object
 2   cat3               1251 non-null   object
 3   brand              1251 non-null   object
 4   sku_size           1251 non-null   int64 
 5   product_id         1251 non-null   int64 
 6   producto_estrella  1251 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 68.5+ KB


In [42]:
df_maestro_prod.head()

,cat1,cat2,cat3,brand,sku_size,product_id,producto_estrella
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280,0
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180,0
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332,0
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222,0
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288,0


In [43]:
prods_desconocidos = set(df_sellout_complet_desde_1er_mes.product_id).difference(set(df_maestro_prod.product_id))
print(len(prods_desconocidos))

45


In [44]:
df_tn_prod_desc = df_sellout_complet_desde_1er_mes[df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos)]
df_tn_prod_desc.tn.sum()

360.2086800000002

In [45]:
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes, df_maestro_prod, on='product_id', how='left')
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.head()

19639107
19639107


,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,meses_historia_prod,meses_historia_cliente,cliente_estrella,prod_cust,cat1,cat2,cat3,brand,sku_size,producto_estrella
0,20524,201701,10234,0,0.0,2.0,0.05300,0.05300,2017-01-01,1,0,0,0,20524-10234,HC,VAJILLA,Cristalino,Importado,500.0,0.0
1,20524,201702,10234,1,0.0,0.0,0.00000,0.00000,2017-02-01,2,1,1,0,20524-10234,HC,VAJILLA,Cristalino,Importado,500.0,0.0
2,20524,201703,10234,0,0.0,1.0,0.01514,0.01514,2017-03-01,3,2,2,0,20524-10234,HC,VAJILLA,Cristalino,Importado,500.0,0.0
3,20524,201704,10234,1,0.0,0.0,0.00000,0.00000,2017-04-01,4,3,3,0,20524-10234,HC,VAJILLA,Cristalino,Importado,500.0,0.0
4,20524,201705,10234,1,0.0,0.0,0.00000,0.00000,2017-05-01,5,4,4,0,20524-10234,HC,VAJILLA,Cristalino,Importado,500.0,0.0


In [46]:
df_sellout_complet_desde_1er_mes.cat1 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat1)
df_sellout_complet_desde_1er_mes.cat2 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat2)
df_sellout_complet_desde_1er_mes.cat3 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat3)
df_sellout_complet_desde_1er_mes.brand = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.brand)
df_sellout_complet_desde_1er_mes.sku_size = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.sku_size)
df_sellout_complet_desde_1er_mes.producto_estrella = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.producto_estrella)

In [47]:
df_exogenas = pd.read_csv(arch_exogenas)
df_exogenas.periodo_fecha = pd.to_datetime(df_exogenas.periodo_fecha)

df_exogenas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periodo_fecha              48 non-null     datetime64[ns]
 1   temp_media                 48 non-null     float64       
 2   temp_max_media             48 non-null     float64       
 3   temp_min_media             48 non-null     float64       
 4   IPC                        48 non-null     float64       
 5   promedio_mens_dolar_venta  48 non-null     float64       
 6   catastrofe                 48 non-null     bool          
dtypes: bool(1), datetime64[ns](1), float64(5)
memory usage: 2.4 KB


In [48]:
# Le agregamos las exogenas
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes,df_exogenas,on="periodo_fecha",how="left")
print(len(df_sellout_complet_desde_1er_mes))

19639107
19639107


In [49]:
# Como control, sumo tns
print("Toneladas Total Control:", round(sum(df_sellout_complet_desde_1er_mes.tn),0),tn_suma_original==round(sum(df_sellout_complet_desde_1er_mes.tn),0))

Toneladas Total Control: 1324989.0 True


In [50]:
df_sellout_complet_desde_1er_mes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19639107 entries, 0 to 19639106
Data columns (total 26 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   product_id                 int64         
 1   periodo                    int64         
 2   customer_id                int64         
 3   cero_ventas                int64         
 4   plan_precios_cuidados      float64       
 5   cust_request_qty           float64       
 6   cust_request_tn            float64       
 7   tn                         float64       
 8   periodo_fecha              datetime64[ns]
 9   mes                        int64         
 10  meses_historia_prod        int64         
 11  meses_historia_cliente     int64         
 12  cliente_estrella           int64         
 13  prod_cust                  object        
 14  cat1                       object        
 15  cat2                       object        
 16  cat3                       object 

In [51]:
# Ordeno para que luego el FE funcione correctamente usando SHIFT
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.sort_values(by=["product_id","customer_id","periodo"],ascending=True)

In [52]:
df_sellout_complet_desde_1er_mes.head(50)

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
2276064,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,2017-01-01,1,...,Liquido,ARIEL,3000.0,1.0,25.65,30.4,20.9,1.6,15.91,False
2276065,20001,201702,10001,0,0.0,23.0,198.84365,198.84365,2017-02-01,2,...,Liquido,ARIEL,3000.0,1.0,25.35,30.1,20.6,2.1,15.59,False
2276066,20001,201703,10001,0,0.0,33.0,92.46537,92.46537,2017-03-01,3,...,Liquido,ARIEL,3000.0,1.0,22.45,26.9,18.0,2.4,15.52,False
2276067,20001,201704,10001,0,0.0,8.0,13.29728,13.29728,2017-04-01,4,...,Liquido,ARIEL,3000.0,1.0,18.90,23.5,14.3,2.7,15.36,False
2276068,20001,201705,10001,0,0.0,15.0,101.20711,101.00563,2017-05-01,5,...,Liquido,ARIEL,3000.0,1.0,15.90,19.5,12.3,1.4,15.72,False
2276069,20001,201706,10001,0,0.0,25.0,128.04792,128.04792,2017-06-01,6,...,Liquido,ARIEL,3000.0,1.0,13.70,17.9,9.5,1.2,16.12,False
2276070,20001,201707,10001,0,0.0,18.0,101.20711,101.20711,2017-07-01,7,...,Liquido,ARIEL,3000.0,1.0,13.35,17.0,9.7,1.7,17.19,False
2276071,20001,201708,10001,0,0.0,17.0,43.33930,43.33930,2017-08-01,8,...,Liquido,ARIEL,3000.0,1.0,15.15,19.2,11.1,1.4,17.42,False
2276072,20001,201709,10001,0,0.0,36.0,289.35024,289.35024,2017-09-01,9,...,Liquido,ARIEL,3000.0,1.0,15.95,20.3,11.6,1.9,18.15,False
2276073,20001,201710,10001,0,0.0,22.0,222.11389,222.11389,2017-10-01,10,...,Liquido,ARIEL,3000.0,1.0,17.15,23.0,11.3,1.5,17.46,False


In [53]:
df_sellout_complet_desde_1er_mes.isna().sum()

product_id                   0
periodo                      0
customer_id                  0
cero_ventas                  0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
tn                           0
periodo_fecha                0
mes                          0
meses_historia_prod          0
meses_historia_cliente       0
cliente_estrella             0
prod_cust                    0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
producto_estrella            0
temp_media                   0
temp_max_media               0
temp_min_media               0
IPC                          0
promedio_mens_dolar_venta    0
catastrofe                   0
dtype: int64

### --> el paso siguiente también puede tardar unos minutos...

In [54]:
# Exportar el DataFrame a un archivo CSV
df_sellout_complet_desde_1er_mes.to_csv(arch_salida_completo, index=False)

## Al final, también se genera un agrupado por producto, por cliente y por categoria
## También se genera un dataset reducido por producto_cliente para pruebas preliminares

In [70]:
df_prod_full = df_sellout_complet_desde_1er_mes.groupby(['product_id','periodo']).agg({
                                                                                    'tn':'sum',
																					'cero_ventas': 'min', #con que alguno de los clientes haya tenido este flag en 0, alcanza para decir que este flag estara en 0
																					'cust_request_qty':'sum',
																					'cust_request_tn':'sum',
                                                                                    'mes':'min',
																					'producto_estrella':'min',
                                                                                    'plan_precios_cuidados':'min',
																					'meses_historia_prod':'min',
																					'cat1':'min',
																					'cat2':'min',
																					'cat3':'min',
																					'sku_size':'min',
																					'temp_media':'min',	
																					'temp_min_media':'min',
																					'temp_max_media':'min',
																					'IPC':'min',	
																					'promedio_mens_dolar_venta':'min',
																					'catastrofe':'min'}).reset_index().sort_values(by=['product_id','periodo'],ascending=True)
df_prod_full = df_prod_full.rename(columns={"tn":"product_id_tn"})
df_prod_full.to_csv(arch_salida_producto,index=False)
len(df_prod_full)

35888

In [56]:
df_cust = df_sellout_complet_desde_1er_mes.groupby(['customer_id','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['customer_id','periodo'],ascending=True)
df_cust = df_cust.rename(columns={"tn":"customer_id_tn"})
df_cust.to_csv(arch_salida_cliente,index=False)
len(df_cust)

19555

In [57]:
df_cat1 = df_sellout_complet_desde_1er_mes.groupby(['cat1','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat1','periodo'],ascending=True)
df_cat1 = df_cat1.rename(columns={"tn":"cat1_tn"})
df_cat1.to_csv(arch_salida_cat1,index=False)
len(df_cat1)

144

In [58]:
df_cat2 = df_sellout_complet_desde_1er_mes.groupby(['cat2','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat2','periodo'],ascending=True)
df_cat2 = df_cat2.rename(columns={"tn":"cat2_tn"})
df_cat2.to_csv(arch_salida_cat2,index=False)
len(df_cat2)

576

In [59]:
df_cat3 = df_sellout_complet_desde_1er_mes.groupby(['cat3','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat3','periodo'],ascending=True)
df_cat3 = df_cat3.rename(columns={"tn":"cat3_tn"})
df_cat3.to_csv(arch_salida_cat3,index=False)
len(df_cat3)

2806

In [61]:
# Control de Toneladas Total
print(df_sellout_complet_desde_1er_mes.tn.sum())
print(df_prod_full.product_id_tn.sum())
print(df_cust.customer_id_tn.sum())
print(df_cat1.cat1_tn.sum())
print(df_cat2.cat2_tn.sum())
print(df_cat3.cat3_tn.sum())

1324988.5884100045
1324988.5884099999
1324988.58841
1324988.5884099999
1324988.5884099999
1324988.5884099999


In [62]:
df_sellout_estrellas = df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.producto_estrella==1) & (df_sellout_complet_desde_1er_mes.cliente_estrella==1)]
df_sellout_estrellas.to_csv(arch_salida_estrellas,index=False)
len(df_sellout_estrellas),len(df_sellout_estrellas.prod_cust.unique())

(4020, 120)

In [69]:
df_prod_full.groupby(["cat1","periodo"])["product_id_tn"].sum()

cat1         periodo
FOODS        201701     6499.75930
             201702     7098.70342
             201703     7617.35140
             201704     7285.05226
             201705     7707.92393
                           ...    
desconocida  201908        0.84865
             201909        0.39541
             201910       25.14985
             201911        0.05425
             201912        0.00000
Name: product_id_tn, Length: 144, dtype: float64

In [67]:
df_cat1.groupby(["cat1","periodo"])["cat1_tn"].sum()

cat1         periodo
FOODS        201701     6499.75930
             201702     7098.70342
             201703     7617.35140
             201704     7285.05226
             201705     7707.92393
                           ...    
desconocida  201908        0.84865
             201909        0.39541
             201910       25.14985
             201911        0.05425
             201912        0.00000
Name: cat1_tn, Length: 144, dtype: float64